In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn


In [ ]:
# Load the original CSV
df = pd.read_csv('dataset.csv')

# Combine date and time into a single timestamp
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

# Sort and save minimal CSV for streaming
df = df.sort_values('Timestamp').reset_index(drop=True)

# Save required columns for Pathway streaming
df[[
    "Timestamp",
    "Occupancy",
    "Capacity",
    "QueueLength",
    "TrafficConditionNearby",
    "IsSpecialDay",
    "VehicleType"
]].to_csv("parking_stream.csv", index=False)


In [ ]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str


In [ ]:
data = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=1000
)


In [ ]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt)
)


In [ ]:
base_price = 10.0
alpha = 1.0         # weight for occupancy ratio
beta = 0.1          # weight for queue length
gamma = 0.5         # weight for traffic level
delta = 0.3         # weight for special day
epsilon = 1.0       # weight multiplier for vehicle type weight
lambda_coeff = 0.5  # controls price sensitivity
min_price = 5.0
max_price = 20.0


In [ ]:
@pw.udf
def vehicle_type_weight(vtype):
    return {
        'car': 0.3,
        'bike': 0.1,
        'truck': 0.5
    }.get(str(vtype).lower(), 0.2)

@pw.udf
def traffic_level_score(tlevel):
    return {
        'low': 0.1,
        'average': 0.5,
        'high': 1.0
    }.get(str(tlevel).lower(), 0.5)


In [ ]:
# UDF for multiplying columns with scalars
@pw.udf
def multiply(x, weight) -> float:
    if x is None:
        return 0.0
    return float(x * weight)



In [ ]:
data_with_demand = data_with_time.with_columns(
    occ_ratio = pw.this.Occupancy / pw.this.Capacity,
    vehicle_score = vehicle_type_weight(pw.this.VehicleType),
    traffic_score = traffic_level_score(pw.this.TrafficConditionNearby)
)


In [ ]:
data_with_demand = data_with_demand.with_columns(
    occ_term = multiply(data_with_demand.occ_ratio, alpha),
    queue_term = multiply(pw.this.QueueLength, beta),
    traffic_term = multiply(data_with_demand.traffic_score, gamma),
    special_term = multiply(pw.this.IsSpecialDay, delta),
    vehicle_term = multiply(data_with_demand.vehicle_score, epsilon)
)


In [ ]:
data_with_demand = data_with_demand.with_columns(
    demand_raw = data_with_demand.occ_term +
                 data_with_demand.queue_term -
                 data_with_demand.traffic_term +
                 data_with_demand.special_term +
                 data_with_demand.vehicle_term
)


In [ ]:
@pw.udf
def normalize_demand(demand):
    return min(1.0, max(0.0, demand / 5.0))  # assume max possible demand ~5.0

data_with_demand = data_with_demand.with_columns(
    demand_norm = normalize_demand(pw.this.demand_raw)
)


In [ ]:
@pw.udf
def compute_price(demand_norm):
    price = base_price * (1 + lambda_coeff * demand_norm)
    return min(max_price, max(min_price, price))

data_with_price = data_with_demand.with_columns(
    price = compute_price(pw.this.demand_norm)
)


In [ ]:
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Dynamic Parking Price (Model 2)",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

viz = data_with_price.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
%%capture --no-display
pw.run()


Output()